In [10]:
library(nnet)
library(party)
library(caret)
library(party)
library(randomForest)
library(e1071)

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich
Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

The following object is masked from ‘package:dplyr’:

    combine



In [56]:
library(dplyr)
bodyPerformance <- read.csv('../../Data/프로젝트/기초대사량.csv')
head(bodyPerformance)
str(bodyPerformance)


나이,성별,키_cm,몸무게_kg,앉아서앞으로구부리기_cm,윗몸일으키기,멀리뛰기,등급,bmr
27,0,172.3,75.24,18.4,60,217,C,1560.5
25,0,165.0,55.80,16.3,53,229,A,1370.2
31,0,179.6,78.00,12.0,49,181,C,1581.4
32,0,174.5,71.10,15.2,53,219,B,1501.3
28,0,173.8,67.70,27.1,45,217,B,1486.2
36,1,165.4,55.40,21.0,27,153,B,1315.4


'data.frame':	13393 obs. of  9 variables:
 $ 나이                   : int  27 25 31 32 28 36 42 33 54 28 ...
 $ 성별                   : int  0 0 0 0 0 1 1 0 0 0 ...
 $ 키_cm                  : num  172 165 180 174 174 ...
 $ 몸무게_kg              : num  75.2 55.8 78 71.1 67.7 ...
 $ 앉아서앞으로구부리기_cm: num  18.4 16.3 12 15.2 27.1 21 0.8 12.3 18.6 12.1 ...
 $ 윗몸일으키기           : num  60 53 49 53 45 27 18 42 34 55 ...
 $ 멀리뛰기               : num  217 229 181 219 217 153 146 234 148 213 ...
 $ 등급                   : Factor w/ 4 levels "A","B","C","D": 3 1 3 2 2 2 4 2 3 2 ...
 $ bmr                    : num  1560 1370 1581 1501 1486 ...


In [57]:
bodyPerformance$등급<-ifelse(bodyPerformance$등급=='A',1,
                           ifelse(bodyPerformance$등급=='B',2,
                                  ifelse(bodyPerformance$등급=='C',3,4)))

In [58]:
str(bodyPerformance)
summary(bodyPerformance)

'data.frame':	13393 obs. of  9 variables:
 $ 나이                   : int  27 25 31 32 28 36 42 33 54 28 ...
 $ 성별                   : int  0 0 0 0 0 1 1 0 0 0 ...
 $ 키_cm                  : num  172 165 180 174 174 ...
 $ 몸무게_kg              : num  75.2 55.8 78 71.1 67.7 ...
 $ 앉아서앞으로구부리기_cm: num  18.4 16.3 12 15.2 27.1 21 0.8 12.3 18.6 12.1 ...
 $ 윗몸일으키기           : num  60 53 49 53 45 27 18 42 34 55 ...
 $ 멀리뛰기               : num  217 229 181 219 217 153 146 234 148 213 ...
 $ 등급                   : num  3 1 3 2 2 2 4 2 3 2 ...
 $ bmr                    : num  1560 1370 1581 1501 1486 ...


      나이            성별            키_cm         몸무게_kg     
 Min.   :21.00   Min.   :0.0000   Min.   :125.0   Min.   : 26.30  
 1st Qu.:25.00   1st Qu.:0.0000   1st Qu.:162.4   1st Qu.: 58.20  
 Median :32.00   Median :0.0000   Median :169.2   Median : 67.40  
 Mean   :36.78   Mean   :0.3678   Mean   :168.6   Mean   : 67.45  
 3rd Qu.:48.00   3rd Qu.:1.0000   3rd Qu.:174.8   3rd Qu.: 75.30  
 Max.   :64.00   Max.   :1.0000   Max.   :193.8   Max.   :138.10  
 앉아서앞으로구부리기_cm  윗몸일으키기      멀리뛰기          등급    
 Min.   :-25.00          Min.   : 0.00   Min.   :  0.0   Min.   :1.0  
 1st Qu.: 10.90          1st Qu.:30.00   1st Qu.:162.0   1st Qu.:2.0  
 Median : 16.20          Median :41.00   Median :193.0   Median :3.0  
 Mean   : 15.21          Mean   :39.77   Mean   :190.1   Mean   :2.5  
 3rd Qu.: 20.70          3rd Qu.:50.00   3rd Qu.:221.0   3rd Qu.:4.0  
 Max.   :213.00          Max.   :80.00   Max.   :303.0   Max.   :4.0  
      bmr        
 Min.   : 888.4  
 1st Qu.:1322.4  
 Median :1

In [60]:
cor(bodyPerformance)

,나이,성별,키_cm,몸무게_kg,앉아서앞으로구부리기_cm,윗몸일으키기,멀리뛰기,등급,bmr
나이,1.00000000,0.06022946,-0.29398048,-0.09996575,-0.07003333,-0.5445811,-0.43517159,0.06561189,-0.5380568
성별,0.06022946,1.00000000,-0.73089939,-0.67288652,0.32746738,-0.4745964,-0.70414892,-0.07560468,-0.6199229
키_cm,-0.29398048,-0.73089939,1.00000000,0.73490928,-0.22196991,0.5004235,0.67458913,0.03775284,0.7959824
몸무게_kg,-0.09996575,-0.67288652,0.73490928,1.00000000,-0.29624858,0.2948988,0.47956401,0.21412856,0.8899447
앉아서앞으로구부리기_cm,-0.07003333,0.32746738,-0.22196991,-0.29624858,1.00000000,0.1771533,0.02648747,-0.58812264,-0.2211394
윗몸일으키기,-0.54458109,-0.47459643,0.50042351,0.29489883,0.17715333,1.0000000,0.74827281,-0.45283180,0.5133511
멀리뛰기,-0.43517159,-0.70414892,0.67458913,0.47956401,0.02648747,0.7482728,1.00000000,-0.26215356,0.6265185
등급,0.06561189,-0.07560468,0.03775284,0.21412856,-0.58812264,-0.4528318,-0.26215356,1.00000000,0.1408584
bmr,-0.53805676,-0.61992286,0.79598237,0.88994466,-0.22113936,0.5133511,0.62651846,0.14085843,1.0000000


# 인공신경망

In [68]:
ANN <- function(tr,x,y){
    model.nnet <- nnet(등급 ~ ., data = tr, size=3)
    # 검증하기
    pred <- predict(model.nnet, x, type = "raw")

    # 예측률 
    return(mean(pred ==y))
}

# 의사결정나무

In [69]:
DMT <- function(tr,x,y){
    model.ctree <- ctree(등급 ~ ., data=tr)
    
    # 검증하기
    pred <- predict(model.ctree, x)
    
    # 예측률 
    return(mean(pred ==y))
}

# RandomForest

In [70]:
RF <- function(tr,x,y){
    model.rf <- randomForest(등급 ~ ., data = tr , ntree=100)

    # 검증하기
    pred <- predict(model.rf, x)    
    # 예측률 
    return(mean(pred ==y))
}

# SVM

In [71]:
SVM <- function(tr,x,y){
    model.svm = svm(등급 ~ ., data=tr)
    # 검증하기
    pred <- predict(model.svm, x)
    # 예측률 
    return(mean(pred ==y))
}

# 로지스틱회귀분석

In [72]:
LRA <- function(tr,x,y){
    model <- multinom(등급 ~ .,data=tr)
    # 검증하기
    pred <- predict(model, x)
    # 예측률
    return(mean(pred ==y))
}

In [73]:
반복수행 <- function(n){
    irr <- iris

    samp <- createDataPartition(bodyPerformance$등급, p=0.7, list=F)

    irrS <- data.frame(scale(bodyPerformance[,-8]))
    irrS$등급 <- bodyPerformance$등급

    tr <- irrS[samp,]
    test <- irrS[-samp,]

    x <- subset(test, select = -등급)
    y <- test$등급

    
    annS <- 0
    dmtS <- 0
    rfS <- 0
    svmS <- 0
    lraS <- 0
    
    for(i in 1:n){
        annS = annS + ANN(tr,x,y)
        dmtS = dmtS + DMT(tr,x,y)
        rfS = rfS + RF(tr,x,y)
        svmS = svmS + SVM(tr,x,y)
        lraS = lraS + LRA(tr,x,y)       
    }
    
    예측률 <- c(annS/n, dmtS/n, rfS/n, svmS/n, lraS/n)
    
    rtn = data.frame(예측률)
    rownames(rtn) = c('인공신경망','의사결정나무','랜덤포레스트','SVM','로지스틱회귀분석')
    
    return(rtn)

}

In [74]:
반복수행(10)

# weights:  31
initial  value 56646.616247 
final  value 32834.000000 
converged


Warning message in randomForest.default(m, y, ...):
“The response has five or fewer unique values.  Are you sure you want to do regression?”

# weights:  40 (27 variable)
initial  value 12999.282224 
iter  10 value 9755.776512
iter  20 value 9466.882240
iter  30 value 8384.298290
final  value 8382.317956 
converged
# weights:  31
initial  value 46631.827663 
final  value 32834.000000 
converged


Warning message in randomForest.default(m, y, ...):
“The response has five or fewer unique values.  Are you sure you want to do regression?”

# weights:  40 (27 variable)
initial  value 12999.282224 
iter  10 value 9755.776512
iter  20 value 9466.882240
iter  30 value 8384.298290
final  value 8382.317956 
converged
# weights:  31
initial  value 46491.458289 
final  value 32834.000000 
converged


...

Warning message in randomForest.default(m, y, ...):
“The response has five or fewer unique values.  Are you sure you want to do regression?”

# weights:  40 (27 variable)
initial  value 12999.282224 
iter  10 value 9755.776512
iter  20 value 9466.882240
iter  30 value 8384.298290
final  value 8382.317956 
converged
# weights:  31
initial  value 41636.568561 
final  value 32834.000000 
converged


Warning message in randomForest.default(m, y, ...):
“The response has five or fewer unique values.  Are you sure you want to do regression?”

# weights:  40 (27 variable)
initial  value 12999.282224 
iter  10 value 9755.776512
iter  20 value 9466.882240
iter  30 value 8384.298290
final  value 8382.317956 
converged


,예측률
인공신경망,0.251494024
의사결정나무,0.124003984
랜덤포레스트,0.001767928
SVM,0.000000000
로지스틱회귀분석,0.600597610
